# 1. Message and Conversation - success !!!

In [2]:
import json
import dotenv
import os
import unittest

dotenv.load_dotenv(".env")
api_key = os.getenv("OPENAI_API_KEY")

class Message:
    """
    Represents a single message in a conversation, which can be a system message,
    an instruction, or a response.

    Attributes:
        role (str): The role associated with the message. Can be 'system', 'user', or 'assistant'.
        content (Any): The content of the message. This can be any data structure.

    Methods:
        _create_message: Internal method to populate the `role` and `content` attributes.
        to_dict: Creates a dictionary representation of the message.
    """

    def __init__(self, role=None, content=None) -> None:
        self.role = role
        self.content = content

    def _create_message(
        self, system=None, response=None, instruction=None, context=None
    ):
        """
        Internal method to set the `role` and `content` attributes based on the provided parameters.

        Args:
            system (str, optional): System-related message. Defaults to None.
            response (dict, optional): Assistant's response. Defaults to None.
            instruction (str, optional): User's instruction. Defaults to None.
            context (dict, optional): Additional context for the instruction. Defaults to None.

        Raises:
            ValueError: If more than one role is provided for a single message.
        """
        if (system and (response or instruction)) or (response and instruction):
            raise ValueError("Error: Message cannot have more than one role.")
        else:
            if response:
                self.role = "assistant"
                self.content = response['content']
            elif instruction:
                self.role = "user"
                self.content = {"instruction": instruction}
                if context:
                    for key, item in context.items():
                        self.content[key] = item
            elif system:
                self.role = "system"
                self.content = system

    def to_dict(self, system=None, response=None, instruction=None, context=None):
        """
        Converts the Message object to a dictionary representation.

        Args:
            system (str, optional): System-related message. Defaults to None.
            response (dict, optional): Assistant's response. Defaults to None.
            instruction (str, optional): User's instruction. Defaults to None.
            context (dict, optional): Additional context for the instruction. Defaults to None.

        Returns:
            dict: A dictionary representation of the Message object.
        """
        self._create_message(
            system=system, response=response, instruction=instruction, context=context
        )
        if self.role == "assistant":
            return {"role": self.role, "content": self.content}
        else:
            return {"role": self.role, "content": json.dumps(self.content)}


class Conversation:
    """
    Manages a conversation consisting of multiple messages.

    Attributes:
        response_counts (int): A class-level counter for responses.
        messages (list): A list of messages in the conversation.
        system (str): System-related message.
        responses (list): A list to store responses.
        message (Message): An instance of the Message class to handle individual messages.

    Methods:
        initiate_conversation: Initializes a new conversation.
        add_messages: Adds a new message to the conversation.
        change_system: Changes the system message.
        append_last_response: Adds the last response to the list of messages.
    """

    response_counts = 0

    def __init__(self, system, messages=[]) -> None:
        self.messages = messages
        self.system = system
        self.responses = []
        self.message = Message()

    def initiate_conversation(self, system, instruction, context=None):
        """
        Initializes a new conversation by setting the initial system and instruction messages.

        Args:
            system (str): The initial system message.
            instruction (str): The initial instruction from the user.
            context (dict, optional): Additional context for the instruction. Defaults to None.
        """
        self.messages = []
        self.add_messages(system=system)
        self.add_messages(instruction=instruction, context=context)

    def add_messages(
        self, system=None, instruction=None, context=None, response=None
    ):
        """
        Adds a new message to the conversation.

        Args:
            system (str, optional): System-related message. Defaults to None.
            instruction (str, optional): User's instruction. Defaults to None.
            context (dict, optional): Additional context for the instruction. Defaults to None.
            response (dict, optional): Assistant's response. Defaults to None.
        """
        message = self.message.to_dict(
            system=system, response=response, instruction=instruction, context=context
        )
        self.messages.append(message)

    def change_system(self, system):
        """
        Changes the initial system message.

        Args:
            system (str): The new system message.
        """
        self.system = system
        self.messages[0] = self.message.to_dict(system=system)

    def append_last_response(self):
        """
        Appends the last response to the list of messages.
        """
        self.add_messages(response=self.responses[-1])


# Correct the test and re-run
class TestMessageAndConversation(unittest.TestCase):

    def test_message_creation(self):
        message = Message()
        self.assertIsNone(message.role)
        self.assertIsNone(message.content)
        
        # Test assistant role and content
        message.to_dict(response={"content": "Hello"})
        self.assertEqual(message.role, "assistant")
        self.assertEqual(message.content, "Hello")
        
        # Test user role and content
        message.to_dict(instruction="tell me a joke", context={"mood": "happy"})
        self.assertEqual(message.role, "user")
        self.assertEqual(message.content, {"instruction": "tell me a joke", "mood": "happy"})
        
        # Test system role and content
        message.to_dict(system="initiate")
        self.assertEqual(message.role, "system")
        self.assertEqual(message.content, "initiate")
        
    def test_conversation_flow(self):
        conversation = Conversation(system="initiate")
        self.assertEqual(conversation.system, "initiate")
        self.assertEqual(conversation.messages, [])
        
        # Test initiate_conversation
        conversation.initiate_conversation(system="initiate", instruction="tell me a joke", context={"mood": "happy"})
        self.assertEqual(len(conversation.messages), 2)
        
        # Test change_system
        conversation.change_system("new system")
        self.assertEqual(conversation.system, "new system")
        self.assertEqual(json.loads(conversation.messages[0]["content"]), "new system")
        
        # Test add_messages and append_last_response
        conversation.add_messages(response={"content": "Here is a joke"})
        self.assertEqual(len(conversation.messages), 3)
        self.assertEqual(conversation.messages[-1]["content"], "Here is a joke")
        
        conversation.responses.append({"content": "Here is another joke"})
        conversation.append_last_response()
        self.assertEqual(len(conversation.messages), 4)
        self.assertEqual(conversation.messages[-1]["content"], "Here is another joke")
        
    def test_invalid_message_creation(self):
        message = Message()
        with self.assertRaises(ValueError):
            message.to_dict(system="initiate", response={"content": "Hello"})

# Run the tests again
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestMessageAndConversation))


...
----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

# 2. API service

# 3. Session